In [11]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
import torch
import torchvision
import cv2
import torchvision.transforms as transforms
import numpy as np
import os
from torch.utils.data import Dataset
from PIL import Image
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.0, 0.0, 0.0), (0.25, 0.25, 0.25))])

In [4]:
sfr_trainset = []        
class UCF101Dataset(Dataset):
    def __init__(self,root,annotation,fpc):
        Array = []
        mid = int(fpc/2)+1
        with open(annotation,'r') as f:
            for line in f:
                Array = [word for line in f for word in line.split()]
        size = int(len(Array)/2)
        for i in range(size):
            if i%2 == 0:
                Array[i] = '/content/drive/My Drive/UCF-101' + '/' + Array[i]
        for i in range(size):
            cap = cv2.VideoCapture(Array[2*i])
            success,image = cap.read()
            count = 1
            success = True
            sfr = []
            while success:
                if count<=fpc:
                    success,image = cap.read()
                    if cv2.waitKey(10) == 27:                     # exit if Escape is hit
                        break
                    if count == mid:
                        tensor_sfr = transform(image)
                        sfr.append(tensor_sfr)
                        sfr.append(int(Array[2*i+1]))
                        sfr_trainset.append(sfr)
                    count +=1                    
                else:
                    break    

UCF101Dataset(root = '/content/drive/My Drive/UCF-101',annotation = '/content/drive/My Drive/UCF-101/trainlist02.txt',fpc = 15)

In [15]:
sfr_testset = []        
class UCF101Dataset(Dataset):
    def __init__(self,root,annotation,fpc):
        Array = []
        mid = int(fpc/2)+1
        with open(annotation,'r') as f:
            for line in f:
                Array = [word for line in f for word in line.split()]
        size = int(len(Array)/2)
        for i in range(size):
            if i%2 == 0:
                Array[i] = '/content/drive/My Drive/UCF-101' + '/' + Array[i]
        for i in range(size):
            cap = cv2.VideoCapture(Array[2*i])
            success,image = cap.read()
            count = 1
            success = True
            sfr = []
            while success:
                if count<=fpc:
                    success,image = cap.read()
                    if cv2.waitKey(10) == 27:                     
                        break
                    if count == mid:
                        tensor_sfr = transform(image)
                        sfr.append(tensor_sfr)
                        sfr.append(int(Array[2*i+1]))
                        sfr_testset.append(sfr)
                    count +=1                    
                else:
                    break    

UCF101Dataset(root = '/content/drive/My Drive/UCF-101',annotation = 'testlist02.txt',fpc = 15)

In [17]:
sfr_testset[25][0].sum()

tensor(398254.5000)

In [0]:
torch.save(sfr_trainset,'./ufc101_sfr_trainset.pt')
torch.save(sfr_testset,'./ufc101_sfr_testset.pt')

In [0]:
sfr_trainset = torch.load('/content/drive/My Drive/UCF-101/ufc101_sfr_trainset.pt')
sfr_testset = torch.load('/content/drive/My Drive/UCF-101/ufc101_sfr_testset.pt')

In [0]:
sfr_trainloader = torch.utils.data.DataLoader(sfr_trainset, batch_size=16,shuffle=True, num_workers=1)
sfr_testloader = torch.utils.data.DataLoader(sfr_testset, batch_size=16, shuffle=True, num_workers=1)
classes = ['Apply Eye Makeup' , 'Apply Lipstick' , 'Archery' , 'Baby Crawling' , 'Balance Beam' , 'Band Marching' , 'Baseball Pitch' , 'Basketball Shooting' , 'Basketball Dunk' , 'Bench Press' , 'Biking' , 'Billiards Shot' , 'Blow Dry Hair' , 'Blowing Candles' , 'Body Weight Squats' , 'Bowling' , 'Boxing Punching Bag' , 'Boxing Speed Bag' , 'Breaststroke' , 'Brushing Teeth' , 'Clean and Jerk' , 'Cliff Diving' , 'Cricket Bowling' , 'Cricket Shot' , 'Cutting In Kitchen' , 'Diving' , 'Drumming' , 'Fencing' , 'Field Hockey Penalty' , 'Floor Gymnastics' , 'Frisbee Catch' , 'Front Crawl' , 'Golf Swing' , 'Haircut' , 'Hammer Throw' , 'Hammering' , 'Handstand Pushups' , 'Handstand Walking' , 'Head Massage' , 'High Jump' , 'Horse Race' , 'Horse Riding' , 'Hula Hoop' , 'Ice Dancing' , 'Javelin Throw' , 'Juggling Balls' , 'Jump Rope' , 'Jumping Jack' , 'Kayaking' , 'Knitting' , 'Long Jump' , 'Lunges' , 'Military Parade' , 'Mixing Batter' , 'Mopping Floor' , 'Nun chucks' , 'Parallel Bars' , 'Pizza Tossing' , 'Playing Guitar' , 'Playing Piano' , 'Playing Tabla' , 'Playing Violin' , 'Playing Cello' , 'Playing Daf' , 'Playing Dhol' , 'Playing Flute' , 'Playing Sitar' , 'Pole Vault' , 'Pommel Horse' , 'Pull Ups' , 'Punch' , 'Push Ups', 'Rafting' , 'Rock Climbing Indoor' , 'Rope Climbing' , 'Rowing' , 'Salsa Spins' , 'Shaving Beard' , 'Shotput' , 'Skate Boarding' , 'Skiing' , 'Skijet' , 'Sky Diving' , 'Soccer Juggling' , 'Soccer Penalty' , 'Still Rings' , 'Sumo Wrestling' , 'Surfing' , 'Swing' , 'Table Tennis Shot' , 'Tai Chi' , 'Tennis Swing' , 'Throw Discus' , 'Trampoline Jumping' , 'Typing' , 'Uneven Bars' , 'Volleyball Spiking' , 'Walking with a dog', 'Wall Pushups' , 'Writing On Board' , 'Yo Yo']

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Single_Frame_Net(nn.Module):
    def __init__(self):
        super(Single_Frame_Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 48, kernel_size = 7, stride = 1, padding = 3)
        self.conv2 = nn.Conv2d(in_channels = 48, out_channels = 64, kernel_size = 7, stride = 1, padding = 3)
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 96, kernel_size = 7, stride = 1, padding = 3)
        self.proj4 = nn.Conv2d(in_channels = 3, out_channels = 96, kernel_size = 7, stride = 1, padding = 3)
        self.bn1 = nn.BatchNorm2d(96)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(in_channels = 96, out_channels = 176, kernel_size = 5, stride = 1, padding = 2)
        self.conv6 = nn.Conv2d(in_channels = 176, out_channels = 256, kernel_size = 5, stride = 1, padding = 2)
        self.proj7 = nn.Conv2d(in_channels = 96, out_channels = 256, kernel_size = 5, stride = 1, padding = 2)
        self.bn2 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(4, 4)
        self.conv8 = nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = 5, stride = 1, padding = 2)
        self.conv9 = nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = 6, stride = 2, padding = 0)
        self.conv10 = nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = 6, stride = 1, padding = 0)
        self.conv11 = nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = 6, stride = 2, padding = 0)
        self.conv12 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 2, stride = 2, padding = 0)
        self.conv13 = nn.Conv2d(in_channels = 256, out_channels = 101, kernel_size = 1, stride = 2, padding = 0)
        

    def forward(self, x):
        x = self.pool1(self.bn1(F.relu(self.conv3(self.conv2(self.conv1(x)))+self.proj4(x))))
        x = self.pool2(self.bn2(F.relu(self.proj7(x)+self.conv6(self.conv5(x)))))
        x = self.conv13(F.relu(self.conv12(self.conv11(self.conv10(self.conv9(self.conv8(x)))))))
        return x

sfr_net = Single_Frame_Net()

In [22]:
sfr_net.to(device)

Single_Frame_Net(
  (conv1): Conv2d(3, 48, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv2): Conv2d(48, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv3): Conv2d(64, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (proj4): Conv2d(3, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (bn1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(96, 176, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv6): Conv2d(176, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (proj7): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv8): Conv2d(256, 384, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv9): 

In [0]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(sfr_net.parameters(), lr=0.0005)#, momentum=0.9)

In [26]:
for epoch in range(3):  # loop over the dataset multiple times
    running_loss = 0.0
    for i,data in enumerate(sfr_trainloader, 0):
        #print(data)
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = sfr_net(inputs).squeeze()
        #print(outputs.size())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

[1,    10] loss: 3.314
[1,    20] loss: 3.210
[1,    30] loss: 3.207
[1,    40] loss: 3.600
[1,    50] loss: 3.569
[1,    60] loss: 3.301
[1,    70] loss: 3.308
[1,    80] loss: 3.355
[1,    90] loss: 3.439
[1,   100] loss: 3.446
[1,   110] loss: 3.204
[1,   120] loss: 3.389
[1,   130] loss: 3.193
[1,   140] loss: 3.260
[1,   150] loss: 3.531
[1,   160] loss: 3.285
[1,   170] loss: 3.302
[1,   180] loss: 3.219
[1,   190] loss: 3.389
[1,   200] loss: 3.056
[1,   210] loss: 3.324
[1,   220] loss: 3.295
[1,   230] loss: 3.136
[1,   240] loss: 3.074
[1,   250] loss: 3.331
[1,   260] loss: 3.181
[1,   270] loss: 3.266
[1,   280] loss: 3.290
[1,   290] loss: 3.167
[1,   300] loss: 3.198
[2,    10] loss: 3.182
[2,    20] loss: 3.040
[2,    30] loss: 2.978
[2,    40] loss: 3.171
[2,    50] loss: 3.077
[2,    60] loss: 3.066
[2,    70] loss: 2.985
[2,    80] loss: 3.078
[2,    90] loss: 3.182
[2,   100] loss: 3.409
[2,   110] loss: 3.194
[2,   120] loss: 3.165
[2,   130] loss: 3.368
[2,   140] 

In [0]:
PATH_SFR = './ucf101_sfr_net_res.pt'
torch.save(sfr_net.state_dict(), PATH_SFR)

In [27]:
PATH_SFR = './ucf101_sfr_net_res.pt'
sfr_net = Single_Frame_Net()
sfr_net.load_state_dict(torch.load(PATH_SFR))

<All keys matched successfully>

In [28]:
sfr_net.to(device)

Single_Frame_Net(
  (conv1): Conv2d(3, 48, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv2): Conv2d(48, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv3): Conv2d(64, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (proj4): Conv2d(3, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (bn1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(96, 176, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv6): Conv2d(176, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (proj7): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv8): Conv2d(256, 384, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv9): 

In [28]:
correct = 0
total = 0
with torch.no_grad():
    for i,data in enumerate(sfr_trainloader,0):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = sfr_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the train images: %d %%' % (100 * correct / total))

Accuracy of the network on the train images: 35 %


In [31]:
correct = 0
total = 0
with torch.no_grad():
    for i,data in enumerate(sfr_testloader,0):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = sfr_net(images).squeeze()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

Accuracy of the network on the test images: 2 %
